In [9]:
import pandas as pd 
import os
from tqdm import tqdm
root_path = "csv/traffic"
drop = ['samples_below_100pct_ff', 'samples_below_95pct_ff', 'samples_below_90pct_ff', 'samples_below_80pct_ff.1',
          'samples_below_80pct_ff', 'samples_below_75pct_ff', 'samples_below_70pct_ff', 'samples_below_65pct_ff',
          'samples_below_60pct_ff', 'samples_below_55pct_ff', 'samples_below_50pct_ff', 'samples_below_45pct_ff',
          'samples_below_40pct_ff', 'samples_below_35pct_ff', 'samples_below_30pct_ff', 'samples_below_25pct_ff',
          'samples_below_20pct_ff', 'samples_below_15pct_ff', 'samples_below_10pct_ff', 'samples_below_5pct_ff']


files = os.listdir(root_path)
for f in tqdm(files):
    df = pd.read_csv('{}/{}'.format(root_path, f))
    try:
        df.drop(columns=drop, inplace=True)
        df.to_csv('{}/{}'.format(root_path, f))
    except ValueError:
        print('Already droped')  ...................................-      
    




  0%|                                                  | 0/194 [00:00<?, ?it/s]
Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\Users\arthur\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\arthur\Anaconda3\lib\site-packages\tqdm\_tqdm.py", line 148, in run
    for instance in self.tqdm_cls._instances:
  File "C:\Users\arthur\Anaconda3\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



Already droped


  1%|▏                                      | 1/194 [01:43<5:31:31, 103.07s/it]

Already droped


100%|█████████████████████████████████████| 194/194 [6:08:57<00:00, 114.11s/it]


In [18]:
for col in df:
    print(df[col].describe())
    print('----------')


count    2.017846e+06
mean     1.008922e+06
std      5.825021e+05
min      0.000000e+00
25%      5.044612e+05
50%      1.008922e+06
75%      1.513384e+06
max      2.017845e+06
Name: Unnamed: 0, dtype: float64
----------
count                 2017846
unique                    288
top       03/03/2018 21:10:00
freq                     9576
Name: utc_time_id, dtype: object
----------
count     2017846
unique          2
top           tmc
freq      1609313
Name: source_ref, dtype: object
----------
count       2017846
unique        30141
top       106+23194
freq            288
Name: source_id, dtype: object
----------
count       2017846
unique            1
top       hereA0106
freq        2017846
Name: feed_id, dtype: object
----------
count    2.017846e+06
mean     9.866922e-01
std      1.145891e-01
min      0.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.000000e+00
Name: primary_link_source_flag, dtype: float64
----------
count    2.017846e+06
mea

## Column analyzing
Analyzing each columns in the data frame, it turned clear that only 3 columns is from our interrest: **utc_time_id, source_id and avg_speed**.



## source_id

The column source_id give us 30141 unique values, refering to 30141 diferrent measurement points through the city in question(San Francisco), futher our dataframe will have the dimensionalite of ** [N, 30141]**, where N is the number of samples, and each column contains the avg_speed of a specific measurement point 

In [19]:
df.head()

,Unnamed: 0,utc_time_id,source_ref,source_id,feed_id,primary_link_source_flag,samples,avg_speed,avg_flow,avg_occ,avg_freeflow_speed,avg_travel_time,high_quality_samples
0,0,03/03/2018 07:30:00,tmc,106-13503,hereA0106,1,5,49.8,NaN,NaN,50.0,0.96,0
1,1,03/03/2018 07:35:00,tmc,106-13503,hereA0106,1,5,50.0,NaN,NaN,50.0,0.96,0
2,2,03/03/2018 07:40:00,tmc,106-13503,hereA0106,1,5,50.0,NaN,NaN,50.0,0.96,0
3,3,03/03/2018 07:45:00,tmc,106-13503,hereA0106,1,5,50.0,NaN,NaN,50.0,0.96,0
4,4,03/03/2018 07:50:00,tmc,106-13503,hereA0106,1,5,50.0,NaN,NaN,50.0,0.96,0


## Time column
The time utc_time_column is in string format, not in datetime format, it is needed to convert for easier use in the future

In [21]:

for f in tqdm(files):
    df = pd.read_csv('{}/{}'.format(root_path, f))
    df = df[['utc_time_id', 'source_id', 'avg_speed']]
    df['utc_time_id'] = pd.to_datetime(df['utc_time_id'], infer_datetime_format=True)
    df.to_csv('{}/{}'.format(root_path, f), index=False)
    


  0%|                                                  | 0/194 [00:00<?, ?it/s]
Exception in thread Thread-8:
Traceback (most recent call last):
  File "C:\Users\arthur\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\arthur\Anaconda3\lib\site-packages\tqdm\_tqdm.py", line 148, in run
    for instance in self.tqdm_cls._instances:
  File "C:\Users\arthur\Anaconda3\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|█████████████████████████████████████| 194/194 [5:49:40<00:00, 108.15s/it]


## File manager

Due the high volume of information that needs to be process, we will create a way to process the data in multiprocess and in diferents computers 

In [1]:
import os
import pickle as pkl
from queue import Queue
import random

class FileManager(object):
    def __init__(self, directory = 'csv/traffic', path = 'C:/Users/arthur/Dropbox/TrafficFlow/lista.pkl'):

        self.files = os.listdir(directory)
        self.done = []
        self.processing = []
        self.path = path
        
    def get_item(self):
        x = self.files.pop(random.randint(0,len(self.files)-1))
        self.processing.append(x)
        save_pkl(self, self.path)
        return x
    def finnish(self, x):
        self.processing.remove(x)
        self.done.append(x)
        save_pkl(self, self.path)
        
    def set_path(self, path):
        self.path = path
        
    def __str__(self):
        msg = '{} itens to be process \n {} itens processed'.format(len(self.files), len(self.processing) + len(self.done))
        return msg
    
def save_pkl(obj, path = 'C:/Users/arthur/Dropbox/TrafficFlow/lista.pkl'):
    pkl.dump(obj, open(path, 'wb'))
    
def load_pkl(path = 'C:/Users/arthur/Dropbox/TrafficFlow/lista.pkl'):
    obj = pkl.load(open(path, 'rb'))
    if len(obj.processing) > 0:
        print('The process was stoped with running process')
        print('Restoring process to process queue')
        obj.files += obj.processing
        obj.processing = []
    return obj
 


In [1]:
dropdox_path = 'C:/Users/arthur/Dropbox/TrafficFlow/lista.pkl'
dir_path = 'csv/traffic'


In [2]:
def get_date(file):
    name, ext = file.rsplit('.')
    return name[-10:]



In [3]:
import pandas as pd
from tqdm import tqdm

def preprocess(file):
    df = pd.read_csv(file, dtype={'source_id':'category'})
    path = 'csv/traffic/{}.csv'.format(get_date(file))
    print(path)
    df_res = df.pivot(index='utc_time_id', columns='source_id', values='avg_speed')
    df_res.to_csv(path, index=True)
    return file
    



In [4]:

class Task(object):
    def __init__(self, file, user, status):
        self.file = file
        self.user = user
        self.status = status 
        

    def get_file(self):
        self.status = 'running'
        return self.file
        
    def __str__(self):
        return '{} - {}'.format(self.user, self.file)
        
def get_status(obj):
    arthur_count = 0
    zaroni_count = 0
    tony_count = 0
    total = len(obj)
    for task in obj:
        if task.status == 'done':
            if task.user == 'arthur':
                arthur_count += 1
            elif task.user == 'zaroni':
                zaroni_count += 1
            else:
                tony_count += 1
    print('Arthur converted {}'.format(arthur_count))
    print('Zaroni converted {}'.format(zaroni_count))
    print('Tony converted {}'.format(tony_count))
    print('{} still missing'.format(total - arthur_count - zaroni_count - tony_count))
    print('Progress: {}/{}'.format((arthur_count+zaroni_count+tony_count),total))

In [5]:
import os 
import pickle as pkl 
direc = os.listdir(dir_path)

create = False

if create:
    len_ = len(direc)
    lista = []
    for i, element in enumerate(direc):
        if i < len_/3:
            lista.append(Task('{}/{}'.format(dir_path, element), 'arthur', 'raw'))
        elif i < len_*2/3:
            lista.append(Task(element, 'Zaroni', 'raw'))
        else:
            lista.append(Task(element, 'Tony', 'raw'))

    pkl.dump(lista, open(dropdox_path, 'wb'))
else:
    lista = pkl.load(open(dropdox_path, 'rb'))


In [8]:
user = 'arthur'


for l in tqdm(lista):
    if l.status == 'raw':
        files_converted = preprocess(l.get_file())
        l.status = 'done'
        pkl.dump(lista, open(dropdox_path, 'wb'))

  0%|                                                  | 0/154 [00:00<?, ?it/s]

csv/traffic/2017_12_11.csv


 47%|███████████████████▏                     | 72/154 [00:49<00:56,  1.44it/s]

csv/traffic/2017_12_12.csv


 47%|███████████████████▍                     | 73/154 [01:44<01:55,  1.42s/it]

...

 99%|█████████████████████████████████████▌| 152/154 [1:00:30<00:47, 23.89s/it]

csv/traffic/2018_03_02.csv


 99%|█████████████████████████████████████▊| 153/154 [1:01:18<00:24, 24.04s/it]

csv/traffic/2018_03_03.csv


100%|██████████████████████████████████████| 154/154 [1:01:53<00:00, 24.12s/it]


In [11]:
for i, l in enumerate(lista):
    print(i, l.file, l.status, l.user)

0 csv/traffic/contracted-Caltrans_text_gn_link_5min_2017_10_01.csv done arthur
1 csv/traffic/contracted-Caltrans_text_gn_link_5min_2017_10_02.csv done arthur
2 csv/traffic/contracted-Caltrans_text_gn_link_5min_2017_10_03.csv done arthur
3 csv/traffic/contracted-Caltrans_text_gn_link_5min_2017_10_04.csv done arthur
4 csv/traffic/contracted-Caltrans_text_gn_link_5min_2017_10_05.csv done arthur
5 csv/traffic/contracted-Caltrans_text_gn_link_5min_2017_10_06.csv done arthur
6 csv/traffic/contracted-Caltrans_text_gn_link_5min_2017_10_07.csv done arthur
7 csv/traffic/contracted-Caltrans_text_gn_link_5min_2017_10_08.csv done arthur
8 csv/traffic/contracted-Caltrans_text_gn_link_5min_2017_10_09.csv done arthur
9 csv/traffic/contracted-Caltrans_text_gn_link_5min_2017_10_10.csv done arthur
10 csv/traffic/contracted-Caltrans_text_gn_link_5min_2017_10_11.csv done arthur
11 csv/traffic/contracted-Caltrans_text_gn_link_5min_2017_10_12.csv done arthur
12 csv/traffic/contracted-Caltrans_text_gn_link_5m

In [7]:
df = pd.read_csv(file, dtype={'source_id':'category'})
path = 'csv/traffic/{}.csv'.format(get_date(file))
print(path)
df_res = df.pivot(index='utc_time_id', columns='source_id', values='avg_speed')





'csv/traffic'

'csv/traffic'